![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Financial Data Structures Study


In [2]:
from datetime import datetime, timedelta
import pandas as pd
qb = QuantBook()

In [6]:
#Add the ES futures contract for this study:
future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Minute,
                        dataNormalizationMode=DataNormalizationMode.BackwardsRatio,
                        dataMappingMode=DataMappingMode.LastTradingDay,
                        contractDepthOffset=0)
#Filter, only consider contract within 6 months of expiry
future.SetFilter(4, 182)
#Range of dates, consider Quantconnect can only handle <6 months of data for minute bars per call
start_date = datetime(2023, 6, 1)
end_date = datetime(2024, 1, 1)

# Split the date range into smaller segments
segments = []
current_start_date = start_date
while current_start_date < end_date:
    current_end_date = min(current_start_date + timedelta(days=30), end_date)
    segments.append((current_start_date, current_end_date))
    current_start_date = current_end_date + timedelta(days=1)

#Initialize a list to hold the dataframe for each segment
segment_dataframes = []

#Process each segment
for start, end in segments:
    #Fetch historical data for the segment
    future_history = qb.GetFutureHistory(future.Symbol, start, end, Resolution.Minute,
                                        fillForward = True, extendedMarketHours = True)
    #Convert FutureHistory object to a DataFrame
    future_history_df = future_history.GetAllData()


    #Append the processed DataFrame to the list
    segment_dataframes.append(future_history_df)

    #Clear the dataframe from memory
    del future_history_df

# Combine all segment Dataframes into a single Dataframe
combined_df = pd.concat(segment_dataframes, ignore_index = False)

print(combined_df.head())

In [12]:
print(combined_df.index)

print(combined_df.index.names)

In [9]:
# Get unique expiry value
unique_expiry_values = combined_df.index.get_level_values('expiry').unique()
print("Unique Expiry Values:")
print(unique_expiry_values)

# Get unique symbol values
unique_symbol_values = combined_df.index.get_level_values('symbol')
print("\nUnique Symbol Values:")
print(unique_symbol_values)